In [31]:
content = """let t = 0; // time variable

function setup() {
    createCanvas(600, 600);
    noStroke();
    fill(40, 200, 40);
}

function preload() {
    console.log("asdfasfd");
}

function draw() {
    background(10, 10); // translucent background (creates trails)

    // make a x and y grid of ellipses
    for (let x = 0; x <= width; x = x + 30) {
        for (let y = 0; y <= height; y = y + 30) {
            // starting point of each circle depends on mouse position
            const xAngle = map(mouseX, 0, width, -4 * PI, 4 * PI, true);
            const yAngle = map(mouseY, 0, height, -4 * PI, 4 * PI, true);
            // and also varies based on the particle's location
            const angle = xAngle * (x / width) + yAngle * (y / height);

            // each particle moves in a circle
            const myX = x + 20 * cos(2 * PI * t + angle);
            const myY = y + 20 * sin(2 * PI * t + angle);

            ellipse(myX, myY, 10); // draw particle
        }
    }

    t = t + 0.01; // update time
}
"""

In [90]:
import pandas as pd
import re


def get_apis() :
    file_path = "./p5_reference.txt"
    df = pd.read_csv(file_path, sep="\n", header=None)
    data = {}
    current_header = ""
    
    for item in df.values.tolist() :
        ele = item[0].strip()
        if(ele[0].isupper() and ele != "JSON") :
            data[ele] = []
            current_header = ele
        else :
            if(ele == "console") :
                print("ELE : ", ele, current_header)
            ele = ele.replace("()", "")
            data[current_header].append(ele)
    
    del data["Foundation"]  
    
#     d = pd.DataFrame().append(data, ignore_index=True)
#     display(d["Foundation"].tolist())

    return data

def get_events() :
    events = ['mouseMoved()', 'mouseDragged()', 'mousePressed()', 'mouseReleased()', 'mouseClicked()', 'doubleClicked()', 'mouseWheel()', 'keyPressed()', 'keyReleased()', 'keyTyped()',  'touchStarted()', 'touchMoved()', 'touchEnded()']
    
    return events


# script_path = "./sample.js"
# with open(script_path, 'r') as f:
#     content = f.readlines()
# content = ''.join(content)

new_content = content
disallowed_symbols = [ "console.log()" "setup", "draw", "preload"]
functions = get_apis()
for key,value in functions.items() :
    for v in value :
        if(v in new_content and v not in disallowed_symbols) :
            search_string = rf"\b(?=\w){v}\b(?!\w)"
            replace_string = "p.{}".format(v)
            new_content = re.sub(search_string, replace_string, new_content)
            
# data = get_apis()
# display(data["Mouse"])
# display(data["Keyboard"])
# display(data["Touch"])

print(new_content)

ELE :  console Foundation
let t = 0; // time variable

function p.setup() {
    p.createCanvas(600, 600);
    p.noStroke();
    p.fill(40, 200, 40);
}

function preload() {
    console.p.log("asdfasfd");
}

function draw() {
    p.background(10, 10); // translucent p.background (creates trails)

    // make a x and y grid of ellipses
    for (let x = 0; x <= p.width; x = x + 30) {
        for (let y = 0; y <= p.height; y = y + 30) {
            // starting p.point of each p.circle depends on mouse position
            const xAngle = p.map(p.mouseX, 0, p.width, -4 * PI, 4 * PI, true);
            const yAngle = p.map(p.mouseY, 0, p.height, -4 * PI, 4 * PI, true);
            // and also varies based on the particle's location
            const angle = xAngle * (x / p.width) + yAngle * (y / p.height);

            // each particle moves in a p.circle
            const myX = x + 20 * p.cos(2 * PI * t + angle);
            const myY = y + 20 * p.sin(2 * PI * t + angle);

            p.elli